# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

## Introduction

[//]: <> (Student provides a high-level overview of the project. Background information such as the problem domain, the project origin, and related data sets or input data is provided.)

This post discusses in detail a solution for the project "Create a Customer Segmentation Report for Arvato Financial Services", which is one of the Data Scientist nanodegree capstone options. It is in fact a continuation of a previous project whose solution was posted [here](https://github.com/bvcmartins/dsndProject3). I chose it because of its broad scope which involves a reasonably complex data cleaning procedure, unsupervised learning, analysis of imbalanced data, and prediction using supervised learning tools. In the following I will discuss the solution and the difficulties faced in some of the most difficult steps.

The objective of this project is to train a model capable of predicting if persons receiving a mailout campaign will become customers of a company. These individuals had been previously selected from the general population on basis of similarity with current customers. 

[//]: <> (The problem which needs to be solved is clearly defined. A strategy for solving the problem, including discussion of the expected solution, has been made.)

The first part of the project applied dimensionality reduction and clustering techniques to segment the general population and to identify the individuals that are most similar to current customers. These individuals were selected for the mailout campaign. 

In the second part, a dataset very similar to the one identified above was divided into train and test sets. The train set contained the reponse variable which states if an individual became a customer after receiving the mailout campaign. Our objective was to train a supervised learning model to predict if an indvidual will become a customer based on a broad variety of features. 

[//]:<> (Metrics used to measure performance of a model or result are clearly defined. Metrics are justified based on the characteristics of the problem.)

The model was tested against the test set through a Kaggle competition and the scores were ranked using the AUC (Area under curve) metric. This metric was used because of the highly imabalanced distribution of the response variable. In the last part of this report we criticize this choice and show why the F1 metric would be more suited to measure model performance.

### The dataset

[//]:<> (Features and calculated statistics relevant to the problem have been reported and discussed related to the dataset, and a thorough description of the input space or input data has been made. Abnormalities or characteristics about the data or input that need to be addressed have been identified.)

Arvato kindly provided us four datasets:

   * Azdias: sample of the general german population categorized according to a variety of features involving personality traits, demographics and financial information (891221 entries, 366 features).
   
   * Customers: classification of current customers according to the same features used for Azdias (191652 entries, 369 features). It also contains customer categorization and information about purchase preferences.
   
   * Mailout_train: training set containing potential customers chosen to receive a mailout campaign  (42962 entries, 367 features). It also contains information if the ad was responded.
   
   * Mailout_test: testing set for the supervised learning model (42833 entries, 366 features).

Two support files for the interpretation of the features were also provided:

   * DIAS Attributes - Values 2017: information about code levels for some of the features.

   * DIAS Information Levels - Attributes 2017: high-level information about most (but not all) features.

Most of the features are ordinal and the numbers represent a label for ranked value levels. Columns marked as float are actually of type int64 mixed with NaN symbols. In fact, NaNs are treated as float64 and, because of its higher precedence, they make the entire column being cast as float. Most recent Pandas versions (higher than 0.24.)0 incorporated the new type Int64 which supports integer NaN. This data type was largely used in this project as a way to reduce the large amount of ancillary code required to control different dtypes.

There are also 6 features of type object. These are categorical variables, except for EINGEFUEGT_AM, which is datetime. This feature was dropped because it did not have any explicative power.

Most of the columns contained NaNs. Actually, NaNs comprised almost 10% of all data. 

### Data cleaning and processing

[//]:<> (Build data visualizations to further convey the information associated with your data exploration journey. Ensure that visualizations are appropriate for the data values you are plotting.)

[//]:<> (All preprocessing steps have been clearly documented. Abnormalities or characteristics about the data or input that needed to be addressed have been corrected. If no data preprocessing is necessary, it has been clearly justified.)

Cleaning this dataset was a relatively complex task mainly because of the number and diversity of features. The steps are outlined below:

* pre-cleaning
* converting missing values to NaN
* assessing missing data per feature
* assessing missing data per row
* imputing missing data
* removing outliers
* re-encoding mixed features
* one-hot encoding categorical features
* scaling numerical features

#### Pre-cleaning

This first cleaning round performed general-purpose operations like converting all numeric features to Int64 (support to integer NaN) and make substitutions of some non-standard missing data codes.

#### Converting missing values to NaN

The challenge with this step was that the missing data codes was feature-dependent. Most of the missing values were coded as -1 or 0 but some of them were coded (and not listed in file DIAS Attributes) as X or XX. They were converted to NaNs during pre-cleaning. 

Missing values coded as -1 or 0 were first converted to a different code (-100) to avoid problems with data type and then later converted to integer NaN.

#### Assessing missing data per feature

After having all missing values converted to NaN we were able to assess which features had more than 445 000 missing. This number is half the total number of entries. As shown below, 9 features satisfying this requirement were found. They corresponded to 18% of all missing values and were all dropped.

![](./figures/missing_feature.png)

#### Assessing missing data per row

In the same way certain columns can have a disproportional amount of missing values, this can also happen to groups of rows. As shown in the figure below, the distribution of missing data per row is multimodal. Most of the rows have less than 50 NaNs but some of them can have at least 180 missing features. These outliers were singled out for further assessment.

![](./figures/missing_row.png)

To decide if these rows should be dropped or imputed the [Kolmogorov-Smirnov test](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test) was applied. This non-parametric test checks if two groups come from the same distribution given a feature. It was aplied to all features that were numeric and that had no missing values in both groups. The null hypothesis was that both groups were identical. In case they are identical, we can impute the missing values. Otherwise we need to drop these rows.

Because the test involved multiple comparisons, the very strict [Bonferroni correction](https://en.wikipedia.org/wiki/Bonferroni_correction) was applied to the p-values. 

Results showed that the difference between the two groups were significant only for 8.2% of the test features. Note that this number is not a p-value and it should not be compared with the 0.05 significance level. The fraction was considered small and the rows were not dropped.

#### Imputing missing data

Imputation was carried out separately for numeric and object variables. Numeric variables were imputed using the [Simple Imputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) with strategy median value. Object-type variables had their missing values replaced by the column mode using pandas [get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) method. 

A few exploratory tests were also performed with the new [Iterative Imputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html) which trains a model to predict the ideal value. K-nearest neighbor was used as the imputing model but the results were similar to conventional Simple Imputer at a much higher computational cost. 

#### Re-encoding mixed features

After removing all NaNs and outliers, next step was to re-encode variables of mixed type. These were:

* PRAEGENDE_JUGENDJAHRE
* CAMEO_INTL_2015
* LP_LEBENSPHASE_FEIN
* LP_LEBENSPHASE_GROB

Variable PLZ8_BAUMAX could also have been reencoded but the explanatory gain was too small. Detailed descriptions of the derived variables and their levels are provided in the [notebook.

#### One-hot encoding

The data was cleaned and variables were reencoded. Next we performed one-hot encoding of all categorical variables. Binary variables were left out. A total of 13 features were transformed.

#### Scaling

At first Standard Scaler was used but it is was too sensitive to outliers. A combination of Robust scaling with outlier Removal led to improved results.

#### Removal of outliers

This step was fundamental to improve the predictor performance in the supervised learning part of the project. After trying to apply a few native methods from sklearn (Isolation Forest, Local Outlier Factor) the simpler method of removing all elements lying out of the inter-quartile range (absolute distance of 3 for a z-transformed distribution) proved to be more effective and easier to implement.  

### Customer Segmentation Report

The objective of the first part of the project was to reduce the dimensionality of the Azdias dataset (sample of general population of Germany) and to categorize the population in a few clusters. The same transformations were applied to the Customers dataset and the clusters with population excess for this group were analyzed.

The analysis comprised the following steps:

1. Reduce dimensionality of the general population dataset using PCA
2. Clusterize the reduced space using K-means in order to identify customer segments
3. Apply the PCA transformation defined in 1 to the customers dataset
4. Clusterize the customer reduced space and identify which cluster have a population excess

#### PCA

After being cleaned, one-hot encoded and scaled, the dataset number of features was increased to 496. Considering that some of these features might be redundant or not important at all, it is desirable for a good training performance to reduce the data dimensionality. I applied PCA to reduce the number of dimension from the original 496 to 107, the minimum number of dimensions needed to explain 80% of the variance. As shown in the scree plot below, despite using 107 dimensions, only 5 are relevant for the explained variance.

![](figures/screeplot_pca.png)

#### K-Means

Next, after the space has had its dimensions reduced, I aplied k-means to generate clusters containing similar instances. K-means is by far the most straightforward clustering approach to this problem and it basically requires only the definition of the number of clusters. 

Choosing the number of clusters requires a trade-off between number of clusters and distance between each element and the cluster center. As shown in the scree plot below, this trade-off is maximized in the point were the kmeans score flatlines and the distance to center is close to minimum. In our case, the optimum point was around 12 clusters. 

It must be pointed out that we could have performed a more formal analysis of the number of clusters using the silhouette score but its high computational cost prevented its application.

![](figures/screeplot_kmeans.png)

With 12 clusters we obatained an approximately balanced cluster population distribution, as shown in the barplot below. This is the baseline distribution for the comparison with the customer cluster occupation.

![](figures/general_pop.png)

We can also acquire some insight about the PCA-cluster process by plotting the projection of the data points on the first two PCA axes. This plot is shown below.

![](figures/general_clusters.png)

We note that the data distribution has a homogeneous oval shape which is not quite separable. The colors identifying different clusters indicate a somewhat arbitrary cluster separation. Probably K-means is not the most adequate method for such a non-separable dataset. DBSCAN would probably perform better here since it would be more sensitive to density variation inside of the data hypervolume.

### Application to Customer dataset

We applied this pipeline to customers, our second dataset, which shows how current company clients are distributed according to the same features present in azdias.

As shown below, the customers dataset shows a population excess in clusters 1 and 9. This result suggests that every person in azdias that falls in these clusters would have a higher probablility of becoming a client.

![](figures/customer_clusters.png)

The PCA component weights for clusters 1 and 9 is shown below. Note how only the first 10 components are relevant for the description of the cluster.

![](figures/kmeans_comps.png)

Given a component we selected the top positive and negative attributes. These two blocks change together and inversely to one another.

Component 3: top 5 positive attributes

1. GEBURTSJAHR
2. D19_VERSAND_ONLINE_QUOTE_12
3. D19_GESAMT_ONLINE_QUOTE_12
4. D19_GESAMT_ANZ_24
5. D19_VERSAND_ANZ_24

Component3: top 5 negative attributes

1. D19_GESAMT_ONLINE_DATUM
2. D19_VERSAND_DATUM 	
3. D19_GESAMT_DATUM
4. KOMBIALTER
5. D19_KONSUMTYP_MAX

Positive attributes depend on age, number of online transactions, transaction activity and MAIL-ORDER transaction activity in the last year. We can suppose that these components are related to online shopping.

Negative attributes depend on actuality of last online transaction and actuality of segment mail-order TOTAL. The last two variables are not in the dictionary. These attributes are harder to interpret but they seem related to actuality of online shopping.

Component 4: top 5 positive attributes

1. KBA13_KW_61_120
2. KBA13_VORB_0
3. KBA13_KMH_210
4. KBA13_SITZE_5 
5. KBA13_CCM_1401_2500

Component 4: top 5 negative attributes

1. KBA13_BJ_1999 	
2. KBA13_KMH_140 	
3. KBA13_SITZE_6
4. KBA13_KW_0_60 
5. KBA13_BJ_2000

All positive and negative features are related to cars. The top 5 are related to larger and newer cars.

Component 5: top 5 positive attributes

1. KOMBIALTER
2. SEMIO_KAEM
3. SEMIO_KRIT
4. KBA13_CCM_0_1400
5. SEMIO_DOM

Component 5: top 5 negative attributes

1. VK_DHT4A
2. KBA13_KW_61_120
3. SEMIO_SOZ
4. HH_EINKOMMEN_SCORE
5. SEMIO_VERT

Positive components are related to personality. They describe someone that is combative, critical-minded and with a dominant personality. These persons also have more potent cars.

Negative components are related to persons with a more community-oriented mind, with lower household incomes and concerned about the environment. These persons also have small cars.

A similar analysis, which we did not carry out, is possible for cluster 9. 

### Supervised Learning Model

The previous part section was aimed at selecting new potential customers that would receive a mailout campaign. The mailout data was split in two approximately equal parts, each with around 43000 entries. One of the blocks is the training set, which contains the same features seen above plus a RESPONSE column which indicates if the person became a customer of the company following the campaign. The other block was used to generate predictions.

Training data has around 43 000 entries and 367 features.

As expected for this kind of study, the response classes are very imbalanced. The classes are:

* 0 - did not become customer after campaign
* 1 - became customer after campaign

Only 1.2% of the entries are of class 1.

If we look at the population distribution per cluster stratifying by response, we observe that they are quite similar to each other and to the customers distribution. This means that:

1. the cluster number does not convey much information for differentiation between classes 0 and 1
2. the elements that received the mailout campaign were indeed selected from the customers analysis. This is a good indication of the consistency of the method.

![](figures/mailout_clusters_1.png)

![](figures/mailout_clusters_0.png)

![](figures/mailout_response.png)

#### Basic Models

Class imblance is a hard problem to deal with. Our first approach to this issue was to use SMOTE to balance the training set. However, this is not without risks. Some important points to keep in mind are:

* SMOTE must only be applied to training set. Predictions on Test set must use the original imbalanced classes
* if inside of a Cross-Validation loop, SMOTE must be calculated for every fold
* We applied SMOTE only to the PCA-transformed dataset because all features are numeric and real. For categorical variables SMOTENC must be used insted
* We performed oversampling of the miniority class. We could also have performed undersampling of the majority class

We first tested different models using their default parameters. For each model we ran 5-fold CV training and testing with and without SMOTE. The results obtained with the test set are shown below.

| Classifier        | NO SMOTE F1 | NO SMOTE AUC | SMOTE F1 | SMOTE AUC  |
| ----------------- |:-----------:|:------------:|:--------:|-----------:|
| AdaBoost          | 0.0         | 0.57         | 0.03     | 0.56       |
| Random Forests    | 0.0         | 0.50         | 0.03     | 0.52       |
| XGBoost           | 0.0         | 0.57         | 0.03     | 0.56       |

Results were not very compelling. The best optimized model, obtained after running the following Grid Search routine, was an AdaBoost Classifier with learning_rate=1.0 and n_estimators=300.

Grid search with SMOTE requires the use of a SMOTE pipeline. Scoring was roc_auc and we used only 3-fold split to save processing time.

```python

ab_model = Pipeline([
        ('sampling', SMOTE()),
        ('ab', AdaBoostClassifier())])

parameters = {"ab__learning_rate" : [0.9, 1.0, 1.1, 2.0],
              "ab__n_estimators": [100, 200, 300, 1000]}

kf = StratifiedKFold(n_splits=3)
grid_ab = GridSearchCV(ab_model, parameters, n_jobs=16, cv=kf, scoring='roc_auc', verbose=10)

grid_ab.fit(X, y)

Results were slightly better this time

| Classifier        | NO SMOTE F1 | NO SMOTE AUC | SMOTE F1 | SMOTE AUC  |
| ----------------- |:-----------:|:------------:|:--------:|-----------:|
| AdaBoost Optimized| 0.0         | 0.55         | 0.04     | 0.58       |

SMOTE was clearly not the best solution for this problem. The dataset also was not helping much. Two modifications greatly improved the results:

1. Use the scale_pos_weight XGBoost parameter for compensation of class imbalance
2. Use the cleaned dataframe instead of the PCA-transformed one

Results were much better now

| Classifier        | F1 | AUC |
| ----------------- |:-----------:|-----------:|
| AdaBoost Optimized| 0.0         | 0.76  |

The XGBClassifier model parameters were:

- base_score=0.5
- booster='gbtree'
- colsample_bylevel=1,
- colsample_bynode=1
- colsample_bytree=0.7
- gamma=0,
- learning_rate=0.01, 
- max_delta_step=0, 
- max_depth=6,
- min_child_weight=1
- n_estimators=500, 
- nthread=None, 
- objective='binary:logistic', 
- random_state=34,
- reg_alpha=0, 
- reg_lambda=1, 
- sample_pos_weight=80,
- scale_pos_weight=1

### Prediction and Kaggle competition 

Prediction of th Kaggle competition dataset resulted in a score of 0.79917. This result seems good at principle but in fact it is very bad. I came to this conclusion after inspecting the confusion matrix obtained for this same model:

| 10418  | 0  |
|:-:|:-:|
|  160   | 0  |

This is telling us that all minority values were misclassified. This is basically just the Naive Classifier. I consider that AUC was not a good metric choice for this problem and F1 would have been more adequate. In fact, I ran grid search optimizing the parameters to maximize F1 instead of AUC and I obtained the following results. 

## Conclusions and next steps

That was a quite interesting, I enjoyed very much the work. Here are the main conclusions from this project:

1. the new integer NaN implemented in Pandas helps a lot with the cleaning
2. outlier removal was very important for result improvement
3. PCA and k-means are good tools for qualitative data analysis, specially if you want to get a feeling of it using visualization. However, for this dataset, they did not led to better predictions
4. Using the XGBoost sample weight parameters was a better solution to class imbalance than SMOTE

A few steps can still be taken to improve this model. A more thorough 